In [2]:
import cv2
import mediapipe as mp
import numpy as np
import wmi


mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils


c = wmi.WMI(namespace='wmi')
methods = c.WmiMonitorBrightnessMethods()[0]

def set_brightness(level):
   
    level = max(0, min(level, 100))
    methods.WmiSetBrightness(level, 0)

def count_fingers(hand_landmarks):
    
    tips_ids = [mp_hands.HandLandmark.THUMB_TIP, 
                mp_hands.HandLandmark.INDEX_FINGER_TIP, 
                mp_hands.HandLandmark.MIDDLE_FINGER_TIP, 
                mp_hands.HandLandmark.RING_FINGER_TIP, 
                mp_hands.HandLandmark.PINKY_TIP]

    fingers = []
    for tip_id in tips_ids:
        if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[tip_id - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)
    return fingers.count(1)

cap = cv2.VideoCapture(0)
paused_frame = None

while cap.isOpened():
    if paused_frame is None:
        ret, frame = cap.read()
    else:
        frame = paused_frame.copy()
        
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

            
            height, width, _ = frame.shape
            thumb_tip_coords = (int(thumb_tip.x * width), int(thumb_tip.y * height))
            index_tip_coords = (int(index_tip.x * width), int(index_tip.y * height))
            cv2.circle(frame, thumb_tip_coords, 10, (255, 0, 0), -1)
            cv2.circle(frame, index_tip_coords, 10, (255, 0, 0), -1)
            cv2.line(frame, thumb_tip_coords, index_tip_coords, (0, 255, 0), 2)

            length = np.linalg.norm(np.array(thumb_tip_coords) - np.array(index_tip_coords))
            brightness_level = int(length)
            set_brightness(brightness_level)

            
            bar_color = (0, int(255 * (brightness_level / 100)), int(255 * ((100 - brightness_level) / 100)))
            cv2.rectangle(frame, (50, 50), (50 + int(3 * brightness_level), 80), bar_color, -1)
            cv2.putText(frame, f'Brightness: {brightness_level}%', (50, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            
            extended_fingers = count_fingers(hand_landmarks)
            

    cv2.imshow("Brightness Controller", frame)
    if cv2.waitKey(1) & 0xFF == 27: 
        break

cap.release()
cv2.destroyAllWindows()
